<a href="https://colab.research.google.com/github/andrew-miao/CS480-680-Project/blob/main/kagglecode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

All code can run on the colab pro. In this Kaggle competition, I propose a new model NewTransformer to solve this task. Additionally, two other ImageText models (use image and text simutaneous), VGGText and MobileRCNN are combined with four interesting models. More details I will update in my github. [link text](https://github.com/andrew-miao/cs680-kaggle).

The directory should be as in the folllowing:

-kagglecode.ipynb

-drive (I mean google drive)

-drive/MyDrive/uw-cs480-fall20.zip

Dependencies:

NLTK

AdaBelief (the install command is in the notebook)

GPU information: NIVIA P100

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip 'drive/MyDrive/uw-cs480-fall20.zip'

Streaming output truncated to the last 5000 lines.
  inflating: uw-cs480-fall20/suffled-images/shuffled-images/2239.jpg  
  inflating: uw-cs480-fall20/suffled-images/shuffled-images/22392.jpg  
  inflating: uw-cs480-fall20/suffled-images/shuffled-images/22550.jpg  
  inflating: uw-cs480-fall20/suffled-images/shuffled-images/22642.jpg  
  inflating: uw-cs480-fall20/suffled-images/shuffled-images/22660.jpg  
  inflating: uw-cs480-fall20/suffled-images/shuffled-images/22705.jpg  
  inflating: uw-cs480-fall20/suffled-images/shuffled-images/22737.jpg  
  inflating: uw-cs480-fall20/suffled-images/shuffled-images/22797.jpg  
  inflating: uw-cs480-fall20/suffled-images/shuffled-images/22799.jpg  
  inflating: uw-cs480-fall20/suffled-images/shuffled-images/22873.jpg  
  inflating: uw-cs480-fall20/suffled-images/shuffled-images/22893.jpg  
  inflating: uw-cs480-fall20/suffled-images/shuffled-images/23029.jpg  
  inflating: uw-cs480-fall20/suffled-images/shuffled-images/23113.jpg  
  inflating: u

In [3]:
pip install adabelief-pytorch==0.1.0

In [1]:
%matplotlib inline
import os
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, utils
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
from PIL import Image
import spacy
import re
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from adabelief_pytorch import AdaBelief
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import math
import torchvision.models as models

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
train_csv = pd.read_csv('./uw-cs480-fall20/train.csv')
test_csv = pd.read_csv('./uw-cs480-fall20/test.csv')
train_csv['text'] = train_csv['season'] + ' ' + train_csv['usage'] + ' ' + train_csv['noisyTextDescription']
test_csv['text'] = test_csv['season'] + ' ' + test_csv['usage'] + ' ' + test_csv['noisyTextDescription']
# Convert category to number
cate2num = {}
num2cate = {}
count = 0
for category in train_csv['category']:
    if category not in cate2num:
        cate2num[category] = count
        num2cate[count] = category
        count += 1

train_size = int(0.8 * len(train_csv))
val_size = int(0.1 * len(train_csv))

np.random.seed(42)
random_idx = np.random.permutation(len(train_csv))
tmp_train_csv = train_csv.iloc[random_idx[:train_size]]
tmp_val_csv = train_csv.iloc[random_idx[train_size:train_size + val_size]]
tmp_test_csv = train_csv.iloc[random_idx[train_size + val_size:]]
tmp_final_train_csv = train_csv.iloc[random_idx[:train_size + val_size]]

tmp_train_csv.to_csv('tmp_train.csv', index=False)
tmp_val_csv.to_csv('tmp_val.csv', index=False)
tmp_test_csv.to_csv('tmp_test.csv', index=False)
tmp_final_train_csv.to_csv('tmp_final_train.csv', index=False)

In [3]:
stopwords = ['nike', 'adidas', 'puma', 'playboy', 'fila', 'french', 'femella', 
             'police', 'q', 'casio', 'doodle', 'tokyo', 'rasasi', 'batman', 'ap',
             'lino', 'perros', 'wrangler', 'rocia', 'baggit', 'paris', 'hilton',
             'elle', 'vii']

In [4]:
from nltk.stem import PorterStemmer

def denoiseTokenize(sentence):
    sentence = re.sub(r"[\*\"“”\n\\…\+\/\=\(\)‘•:\[\]\|’\!;><]", " ", str(sentence))
    sentence = re.sub(r"[ ]+", " ", sentence)
    sentence = re.sub(r"\!+", "!", sentence)
    sentence = re.sub(r"\,+", ",", sentence)
    sentence = re.sub(r"\?+", "?", sentence)
    sentence = re.sub(r"\&+", " ", sentence)
    sentence = re.sub(r"\'s", " ", sentence)
    sentence = sentence.lower()
    
    ps = PorterStemmer()
    tokens = []
    for token in word_tokenize(sentence):
        no_add = False
        if token != ' ':
            for i in range(10):
                if str(i) in token:
                    no_add = True
            if not no_add and token not in stopwords:
                token = ps.stem(token)
                tokens.append(token)
    return tokens

In [5]:
def statsCorpus(csv_file, train=True):
    corpus = [None] * len(csv_file)
    stats = {}
    for i in range(len(csv_file)):
        sentence = csv_file['text'].iloc[i]
        corpus[i] = denoiseTokenize(sentence)
        for token in corpus[i]:
            if token in stats:
                stats[token] += 1
            else:
                stats[token] = 1
    
    if not train:
        return corpus
    
    return corpus, stats

In [6]:
def reconstructCorpus(csv_file, train=True, stats=None):
    if not train and stats is None:
        raise ValueError('val/test file should has stats')

    if stats is None:
        corpus, stats = statsCorpus(csv_file)
    else:
        corpus = statsCorpus(csv_file, train=False)

    for i in range(len(csv_file)):
        sentence = ''
        for token in corpus[i]:
            if token in stats:
                if stats[token] > 2:
                    sentence += token + ' '
            
        csv_file['text'].iloc[i] = sentence
    
    if not train:
        return csv_file
        
    return csv_file, stats

In [7]:
tmp_train_csv, train_stats = reconstructCorpus(tmp_train_csv)
tmp_val_csv = reconstructCorpus(tmp_val_csv, train=False, stats=train_stats)
tmp_test_csv = reconstructCorpus(tmp_test_csv, train=False, stats=train_stats)
test_csv = reconstructCorpus(test_csv, train=False, stats=train_stats)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [8]:
tmp_train_csv

,id,category,gender,baseColour,season,usage,noisyTextDescription,text
383,44183,Topwear,Women,Navy Blue,Fall,Casual,Arrow Woman Elephant Baroque Navy Vomero-5 Swe...,fall casual arrow woman eleph baroqu navi swea...
6186,19264,Bags,Women,Red,Summer,Casual,Lino Perros LP226-C3 VII Handbag,summer casual handbag
8877,10613,Jewellery,Women,Gold,Summer,Casual,Lucera Gold Plated Earrings,summer casual lucera gold plate ear
843,6068,Flip Flops,Men,Black,Summer,Casual,Puma Men Glam Art Black Flip Flops,summer casual men glam art black flip flop
9619,34484,Topwear,Men,Navy Blue,Fall,Casual,Flame Delight Solid Navy Blue Sweatshirts,fall casual flame solid navi blue sweatshirt
...,...,...,...,...,...,...,...,...
10234,5974,Shoes,Men,Black,Summer,Formal,SHE-3801L-7ADR Men Black Leather Formal Shoes,summer formal men black leather formal shoe
4763,7655,Fragrance,Women,Black,Spring,Casual,Paris Hilton Women Heiress Thrill,spring casual women heiress thrill
9062,16251,Watches,Men,Black,Winter,Casual,SKAGEN Strike Banks Black Dial Watch 861XLSXB,winter casual skagen strike bank black dial wa...
1874,15681,Bags,Women,Black,Summer,Casual,French Connection Women Black Beaded Clutch,summer casual connect women black bead clutch


In [9]:
tmp_val_csv

,id,category,gender,baseColour,season,usage,noisyTextDescription,text
13682,46596,Topwear,Men,White,Summer,Casual,ADIDAS Men's Strip Kollege BW335,summer casual men strip kolleg
20854,17563,Topwear,Women,Black,Summer,Casual,Domyos Fluroscent Hers Black T-shirt,summer casual domyo flurosc her black t-shirt
5831,15335,Topwear,Girls,Pink,Summer,Casual,Gini Texture Jony Kiss Pretty Blossom Sage Top,summer casual gini textur joni kiss pretti blo...
19530,28321,Fragrance,Men,Black,Spring,Casual,Fogg Fatale Marco Mini,spring casual fogg mini
191,29129,Makeup,Women,Skin,Spring,Casual,BGSW02 Metro to Five Flawless Creme Shell Compact,spring casual to five flawless creme shell com...
...,...,...,...,...,...,...,...,...
4837,32487,Topwear,Women,Black,Summer,Ethnic,Finish Dicolor Black & White Kurta,summer ethnic finish dicolor black white kurta
18677,59555,Wallets,Women,Purple,Winter,Casual,750 Women SD Taj Purple Wallet,winter casual women sd taj purpl wallet
6548,3955,Belts,Women,Multi,Summer,Casual,Fossil Pin Multi Coloured Quilted Squares Belt,summer casual fossil pin multi colour quilt sq...
4481,2661,Shoes,Women,Turquoise Blue,Winter,Casual,Catwalk Women Ballerina Citi Casual Shoe,winter casual catwalk women ballerina citi cas...


In [10]:
tmp_test_csv

,id,category,gender,baseColour,season,usage,noisyTextDescription,text
10837,46086,Lips,Women,Purple,Spring,Casual,Colorbar Os Sweep Mulberry Lipstick Kapteren,spring casual colorbar os lipstick kapteren
6140,12322,Fragrance,Men,Blue,Spring,Ethnic,Mont Blanc Shine Zephyral Boss,spring ethnic mont blanc shine boss
9956,49857,Shoes,Men,Grey,Fall,Casual,Flying Machine Men Grey Casual Shoes,fall casual fli machin men grey casual shoe
1549,55845,Bottomwear,Girls,Blue,Summer,Casual,Gini and A555 Girls Blue Shorts,summer casual gini and girl blue short
14818,46890,Topwear,Men,White,Summer,Casual,UCB Men Ben Logo Chest White Shirt,summer casual ucb men ben logo chest white shirt
...,...,...,...,...,...,...,...,...
11964,16939,Wallets,Men,Brown,Summer,Casual,Hanawi Men Brown Wanigan,summer casual hanawi men brown wanigan
21575,50422,Bottomwear,Women,Pink,Fall,Casual,Forever New Women Blossom M075J002Y Bad Skirt,fall casual forev new women blossom skirt
5390,8967,Topwear,Women,Blue,Summer,Casual,Tokyo Talkies Women Blue Mousie,summer casual talki women blue
860,44917,Topwear,Women,Multi,Summer,Ethnic,Diva Women Multi Bandana Kurta,summer ethnic diva women multi kurta


In [11]:
test_csv

,id,gender,baseColour,season,usage,noisyTextDescription,text
0,26266,Men,Black,Summer,Casual,Chromozome Men Black Fashion Vest,summer casual chromozom men black fashion vest
1,22134,Women,Green,Summer,Casual,Elle Women Green Color Clash Top,summer casual women green color clash top
2,28358,Women,Black,Winter,Casual,Baggit Women Chotu Mayur Black Palms,winter casual women chotu mayur black palm
3,15554,Men,Black,Fall,Casual,Greensboro Colors Of Ap Men Latnam Black Casua...,fall casual color of men latnam black casual s...
4,53408,Women,Purple,Summer,Casual,Fish White Smooth daddy (SH100),summer casual white smooth daddi
...,...,...,...,...,...,...,...
21623,39737,Men,Brown,Winter,Casual,Park ClubLife 602 Helium Sunglasses,winter casual park sunglass
21624,57477,Women,Black,Summer,Casual,Vero Moda Women Black Rey Shift Dress,summer casual vero moda women black rey shift ...
21625,22312,Men,White,Fall,Sports,Nike Men Hell With White Tshirts,fall sport men hell with white tshirt
21626,54105,Women,Red,Spring,Casual,Lakme Holi to Five Day Apricot Nectar Lip K9,spring casual lakm to five day lip


In [12]:
class Tokenize(object):
    def __init__(object):
        pass
    
    def tokenizer(self, sentence):
        tokens = []
        for tok in word_tokenize(sentence):
            if tok != ' ' and tok != ',' and tok != '.':
                tokens.append(tok)
        # return list(nltk.bigrams(tokens))
        return tokens

In [13]:
def stats_sentence(sentence, vocab2num, count, max_seq):
    for i in range(len(sentence)):
        if sentence[i] not in vocab2num:
            vocab2num[sentence[i]] = count
            count += 1

    max_seq = max(len(sentence), max_seq)
    return vocab2num, count, max_seq

In [14]:
def stats_data(data, train=True):
    raw_src_data = [None] * len(data)
    src_tokenize = Tokenize()
    src_vocab2num = {'<unk>': 0, '<pad>': 1}
    src_count = 2
    max_src_seq = 0
    for i in range(len(data)):
        raw_src_data[i] = src_tokenize.tokenizer(data['text'].iloc[i])
        if train:
            src_vocab2num, src_count, max_src_seq = stats_sentence(raw_src_data[i], src_vocab2num, src_count, max_src_seq)

    if not train:
        return raw_src_data

    return raw_src_data, src_vocab2num, max_src_seq

In [15]:
class Products(Dataset):
    def __init__(self, csv_file, raw_src, src_vocab2num, max_seq,
                 cate2num=cate2num, root_dir='./uw-cs480-fall20/suffled-images/shuffled-images/', 
                 transform=None, real_test=False):
        self.product_csv = pd.read_csv(csv_file)
        self.raw_src = raw_src
        self.src_vocab2num = src_vocab2num
        self.max_seq = max_seq
        self.cate2num = cate2num
        self.root_dir = root_dir
        self.transform = transform
        self.real_test = real_test
        self.src_data = torch.ones(len(self.raw_src), self.max_seq, dtype=torch.long)
        self.build_text_tensor()
        
    def  build_text_tensor(self):
        for i in range(len(self.raw_src)):
            for j in range(min(len(self.raw_src[i]), max_seq)):
                word = self.raw_src[i][j]
                if word in self.src_vocab2num:
                    self.src_data[i][j] = self.src_vocab2num[word]
                else:
                    self.src_data[i][j] = self.src_vocab2num['<unk>']

    def __len__(self):
        return len(self.product_csv)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        img_path = str(self.product_csv['id'].iloc[idx]) + '.jpg'
        img_path = os.path.join(self.root_dir, img_path)
        image = Image.open(img_path).convert('RGB')
        src_text = self.src_data[idx]
        if not self.real_test:
            category = self.product_csv['category'].iloc[idx]
            label = self.cate2num[category]
        if self.transform:
            image = self.transform(image)
        if self.real_test:
            return image, src_text

        return image, src_text, label

In [16]:
raw_train_data, train_vocab2num, max_seq= stats_data(tmp_train_csv)
raw_val_data = stats_data(tmp_val_csv, train=False)
raw_test_data = stats_data(tmp_test_csv, train=False)
raw_real_test_data = stats_data(test_csv, train=False)

In [17]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
train_transform = transforms.Compose([transforms.Resize(256),
                                      transforms.RandomResizedCrop(224), 
                                      transforms.RandomHorizontalFlip(), 
                                      transforms.ToTensor(),
                                      normalize])
test_transform = transforms.Compose([transforms.Resize(256),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     normalize])

In [18]:
train = Products('tmp_train.csv', raw_train_data, train_vocab2num, max_seq, transform=train_transform)
val = Products('tmp_val.csv', raw_val_data, train_vocab2num, max_seq, transform=test_transform)
test = Products('tmp_test.csv', raw_test_data, train_vocab2num, max_seq, transform=test_transform)
real_test = Products('./uw-cs480-fall20/test.csv', raw_real_test_data, train_vocab2num, max_seq, transform=test_transform, real_test=True)

In [19]:
max_seq

14

In [20]:
batch_size = 50
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val, batch_size=10, shuffle=False)
test_loader = DataLoader(test, batch_size=10, shuffle=False)
real_test_loader = DataLoader(real_test, batch_size=10, shuffle=False)

In [21]:
def conv3x3(in_planes, out_planes, stride=1, padding=1, dilation=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=padding, dilation=dilation, bias=False)

def conv1x1(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)

In [22]:
class ResBasicBlock(nn.Module):    
    def __init__(self, in_planes, out_planes, stride=1, downsample=None, norm_layer=None):
        super(ResBasicBlock, self).__init__()

        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self.conv1 = conv3x3(in_planes, out_planes, stride)
        self.bn1 = norm_layer(out_planes)
        self.conv2 = conv3x3(out_planes, out_planes)
        self.bn2 = norm_layer(out_planes)
        self.downsample = downsample
        self.relu = nn.ReLU(inplace=True)
    
    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample is not None:
            identity = self.downsample(x)
        out += identity
        out = self.relu(out)
        return out

In [23]:
class ResNet(nn.Module):
    def __init__(self, block, layers, norm_layer=None, expansion=1):
        super(ResNet, self).__init__()
        if norm_layer is None:
            self.norm_layer = nn.BatchNorm2d
        else:
            self.norm_layer = norm_layer
        self.expansion = expansion
        self.in_planes = 64
        self.conv1 = nn.Conv2d(3, self.in_planes, 7, stride=2, padding=3, bias=False)
        self.bn1 = self.norm_layer(self.in_planes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self.makelayer(block, 64, layers[0], stride=1)
        self.layer2 = self.makelayer(block, 128, layers[1], stride=2)
        self.layer3 = self.makelayer(block, 256, layers[2], stride=2)
        self.layer4 = self.makelayer(block, 512, layers[3], stride=2)
        self.globalAvg = nn.AdaptiveAvgPool2d((1, 1))
    
    def makelayer(self, block, planes, num_blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_planes != planes * self.expansion:
            downsample = nn.Sequential(conv1x1(self.in_planes, planes * self.expansion, stride),
                                       self.norm_layer(planes * self.expansion))
        layer = [block(self.in_planes, planes, stride=stride, downsample=downsample, norm_layer=self.norm_layer)]
        self.in_planes = planes * self.expansion
        for i in range(1, num_blocks):
            layer.append(block(self.in_planes, planes, norm_layer=self.norm_layer))
        return nn.Sequential(*layer)
    
    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.maxpool(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.globalAvg(out)
        out = torch.flatten(out, 1)
        return out

In [24]:
import time

def timeSince(since):
    now = time.time()
    s = now - since
    m = s // 60
    s -= m * 60
    return '%dm %ds' % (m, s)

In [25]:
class ImageModel(nn.Module):
    def __init__(self, pretrain_model, output_size=27):
        super(ImageModel, self).__init__()

        self.pretrain_model = pretrain_model
        self.fc1 = nn.Linear(1000, 256)
        self.fc2 = nn.Linear(256, output_size)
        self.relu = nn.ReLU()

    def forward(self, image):
        output = self.pretrain_model(image)
        output = self.relu(self.fc1(output))
        return self.fc2(output)

In [26]:
def evaluateImageModel(model, data, criterion, device):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for image, _, trg in data:
            image, trg = image.to(device), trg.to(device)
            output = model(image)
            pred = torch.argmax(output, dim=1)
            correct += (pred == trg).sum().item()
            total += len(trg)
            loss = criterion(output, trg)
            total_loss += loss.item()

    return total_loss / len(data), correct / total

In [27]:
def trainImageModel(model, train_data, val_data, n_epochs, criterion, optimizer, device, path, data_aug=0.5):
    model.train()
    train_acc_list = []
    val_acc_list = []
    max_acc = None
    step = 0
    print_every = len(train_data)
    image1 = None
    noise_data = False
    start = time.time()
    for epoch in range(n_epochs):
        running_loss = 0.0
        running_correct = 0
        running_total = 0
        for image, _, trg in train_data:
            optimizer.zero_grad()
            step += 1
            image, trg = image.to(device),  trg.to(device)
            output = None
            tmp = np.random.uniform(0, 1, 1)
            if tmp < data_aug:
                nosie_data = True
            else:
                noise_data = False
            if noise_data:
                if image1 is None:
                    image1 = image
                else:
                    if image1.size(0) == image.size(0):
                        new_image = 0.05 * image1 + 0.95 * image
                    else:
                        new_image = 0.05 * image1[:image.size(0)] + 0.95 * image
                    image1 = image
                    image = torch.cat((image, new_image), dim=0)
                    trg = torch.cat((trg, trg), dim=0)
            output = model(image)
            pred = torch.argmax(output, dim=1)
            running_correct += (pred == trg).sum().item()
            running_total += len(trg)
            loss = criterion(output, trg)
            running_loss += loss.item()
            loss.backward()
            optimizer.step()

            if step % print_every == 0:
                val_loss, val_acc = evaluateImageModel(model, val_data, criterion, device)
                print(('%d/%d (%s) train loss: %.3f, train acc: %.2f%%, val loss: %.3f, val acc: %.2f%%') %
                      (epoch + 1, n_epochs, timeSince(start), running_loss / len(train_data), 
                       100 * running_correct / running_total, val_loss, 100 * val_acc))
                train_acc_list.append(running_correct / running_total)
                val_acc_list.append(val_acc)
                if max_acc is None or max_acc < val_acc:
                    if max_acc is None:
                        print(('Validation accuracy in first epoch: %.2f%%') % (100 * val_acc))
                    else:
                        print(('Validation accuracy is increasing: %.2f%% --> %.2f%%') % 
                              (100 * max_acc, 100 * val_acc))
                    max_acc = val_acc
                    print('Saving model...')
                    torch.save(model, path)
                
                model.train()
                running_loss = 0.0
                running_correct = 0
                running_total = 0
    
    return train_acc_list, val_acc_list

In [ ]:
import torchvision.models as models

model_list = [ImageModel(models.googlenet(pretrained=True)), ImageModel(models.shufflenet_v2_x1_0(pretrained=True))]
model_path = ['googlenet.pt', 'shufflenet.pt']
model_name = ['GoogleNet', 'ShuffleNet']
criterion = nn.CrossEntropyLoss()
n_epochs = 50
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
for i, model in enumerate(model_list):
    model.to(device)
    path = model_path[i]
    optimizer = AdaBelief(model.parameters())
    train_acc_list, val_acc_list = trainImageModel(model, train_loader, val_loader,
                                                   n_epochs, criterion, optimizer,
                                                   device, path, data_aug=0.7)
    model = torch.load(path)
    _, test_acc = evaluateImageModel(model, test_loader, criterion, device)
    print('%s accuracy in test set: %.2f%%' % (model_name[i], 100 * test_acc))

model_list = [torch.load('googlenet.pt'), torch.load('shufflenet.pt')]
n_epochs = 30
for i, model in enumerate(model_list):
    model.to(device)
    path = model_path[i]
    optimizer = optim.SGD(model.parameters(), lr=3e-04, momentum=0.9)
    train_acc_list, val_acc_list = trainImageModel(model, train_loader, val_loader,
                                                   n_epochs, criterion, optimizer,
                                                   device, path, data_aug=0)
    model = torch.load(path)
    _, test_acc = evaluateImageModel(model, test_loader, criterion, device)
    print('%s accuracy in test set: %.2f%%' % (model_name[i], 100 * test_acc))

Please check your arguments if you have upgraded adabelief-pytorch from version 0.0.5.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  ---------
adabelief-pytorch=0.0.5  1e-08  False              False
Current version (0.1.0)  1e-16  True               True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer

Weight decoupling enabled in AdaBelief
Rectification enabled in AdaBelief
1/50 (1m 38s) train loss: 1.117, train acc: 69.63%, val loss: 0.531, val acc: 83.95%
Validation accuracy in first epoch: 83.95%
Saving model...
2/50 (3m 17s) train loss: 0.757, train acc: 78.75%, val loss: 0.465, val acc: 87.14%
Validation accuracy is increasing: 83.95% --> 87.14%
Saving model...
3/50 (4m 56s) train loss: 0.698, train acc: 79.97%, val loss: 0.470, val acc: 86.68%
4/50 (6m 35s) train loss: 0.688, train acc: 80.54%, val loss: 0.537, val ac

In [28]:
def evaluateImageText(model, data, criterion, device):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for image, src, trg in data:
            image, src, trg = image.to(device), src.to(device), trg.to(device)
            output = model(image, src)
            pred = torch.argmax(output, dim=1)
            correct += (pred == trg).sum().item()
            total += len(trg)
            loss = criterion(output, trg)
            total_loss += loss.item()

    return total_loss / len(data), correct / total

In [29]:
def trainImageText(model, train_data, val_data, n_epochs, criterion, optimizer, device, path, data_aug=0.5, fusion_ratio=0.25):
    model.train()
    train_acc_list = []
    val_acc_list = []
    max_acc = None
    step = 1
    print_every = len(train_data)
    aug_flag = False
    last_image = None
    last_src = None
    start = time.time()
    for epoch in range(n_epochs):
        running_loss = 0.0
        running_correct = 0
        running_total = 0
        for image, src, trg in train_data:
            optimizer.zero_grad()
            step += 1
            image, src, trg = image.to(device), src.to(device), trg.to(device)
            tmp = np.random.uniform(0, 1, 1)
            if tmp < data_aug:
                aug_flag = True
            else:
                aug_flag = False
            if aug_flag:
                if last_image is None:
                    last_image = image
                    last_src = src
                else:
                    if last_image.size(0) == image.size(0):
                        new_image = fusion_ratio * last_image + (1 - fusion_ratio) * image
                    else:
                        new_image = fusion_ratio * last_image[:image.size(0)] + (1 - fusion_ratio) * image
                    last_image = image
                    image = torch.cat((image, new_image), dim=0)
                    src = torch.cat((src, src), dim=0)
                    trg = torch.cat((trg, trg), dim=0)
            output = model(image, src)
            pred = torch.argmax(output, dim=1)
            running_correct += (pred == trg).sum().item()
            running_total += len(trg)
            loss = criterion(output, trg)
            running_loss += loss.item()
            loss.backward()
            optimizer.step()
            if step % print_every == 0:
                val_loss, val_acc = evaluateImageText(model, val_data, criterion, device)
                print(('%d/%d (%s) train loss: %.3f, train acc: %.2f%%, val loss: %.3f, val acc: %.2f%%') %
                      (epoch + 1, n_epochs, timeSince(start), running_loss / len(train_data), 
                       100 * running_correct / running_total, val_loss, 100 * val_acc))
                train_acc_list.append(running_correct / running_total)
                val_acc_list.append(val_acc)
                if max_acc is None or max_acc < val_acc:
                    if max_acc is None:
                        print(('Validation accuracy in first epoch: %.2f%%') % (100 * val_acc))
                    else:
                        print(('Validation accuracy is increasing: %.2f%% --> %.2f%%') % 
                              (100 * max_acc, 100 * val_acc))
                    max_acc = val_acc
                    print('Saving model...')
                    torch.save(model, path)
                
                model.train()
                running_loss = 0.0
                running_correct = 0
                running_total = 0
    
    return train_acc_list, val_acc_list

In [30]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model=512, dropout=0.1, max_len=15):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [31]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [32]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model=512):
        super(EncoderLayer, self).__init__()

        self.d_model = d_model
        self.softmax = nn.Softmax(dim=2)

    def attention(self, query, key, value):
        attn_weights = self.softmax(torch.bmm(query, key.permute(0, 2, 1)) / self.d_model)
        return torch.bmm(attn_weights, value)

    def forward(self, x):
        residual = x
        x = self.attention(x, x, x)
        return x + residual

In [33]:
class NewTransformer(nn.Module):
    def __init__(self, n_vocab, block, layers, n_encoders=1, output_size=27, max_seq=15, d_model=512, dropout=0.1):
        super(NewTransformer, self).__init__()
        
        self.d_model = d_model
        self.resnet = ResNet(block, layers)
        self.position_encoder = PositionalEncoding(d_model, max_len=max_seq, dropout=dropout)
        self.embedding = nn.Embedding(n_vocab + 1, d_model)
        self.softmax = nn.Softmax(dim=2)
        self.fc1 = nn.Linear(d_model, 128)
        self.fc2 = nn.Linear(128, output_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.encoder_blocks = nn.ModuleList([EncoderLayer(d_model=d_model) for _ in range(n_encoders)])

    def attention(self, query, key, value):
        attn_weights = self.softmax(torch.bmm(query, key.permute(0, 2, 1)) / self.d_model)
        return torch.bmm(attn_weights, value)

    def encoder(self, image, src):
        image = self.resnet(image)
        image = torch.unsqueeze(image, dim=1)
        src = self.embedding(src)
        encoder_output = self.position_encoder(src) + image
        for block in self.encoder_blocks:
            encoder_output = block(encoder_output)
        return image, encoder_output

    def decoder(self, image, encoder_out):
        target = 27 * torch.ones(encoder_out.size(0), 1, dtype=torch.long).to(device)
        target = self.embedding(target)
        output = self.attention(target, encoder_out, encoder_out)
        output.add_(image)
        output = self.relu(self.fc1(output))
        output = self.dropout(output)
        output = self.fc2(output)
        return torch.squeeze(output, dim=1)
    
    def forward(self, image, src):
        image, encoder_out = self.encoder(image, src)
        return self.decoder(image, encoder_out)

In [57]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NewTransformer(len(train_vocab2num), ResBasicBlock, [3, 4, 6, 3], n_encoders=1, max_seq=max_seq).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = AdaBelief(model.parameters(), lr=3e-04)
path = 'new_transformer.pt'
n_epochs = 50
train_acc_list, val_acc_list = trainImageText(model, train_loader, val_loader, n_epochs, criterion, optimizer, device, path, data_aug=0.6, fusion_ratio=0.05)
model = torch.load(path)
_, test_acc = evaluateImageText(model, test_loader, criterion, device)
print('Accuracy in test set: %.2f%%' % (100 * test_acc))
n_epochs = 20
model = torch.load(path)
optimizer = AdaBelief(model.parameters(), lr=1e-04)
train_acc_list, val_acc_list = trainImageText(model, train_loader, val_loader, n_epochs, criterion, optimizer, device, path, data_aug=0)
model = torch.load(path)
_, test_acc = evaluateImageText(model, test_loader, criterion, device)

Please check your arguments if you have upgraded adabelief-pytorch from version 0.0.5.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  ---------
adabelief-pytorch=0.0.5  1e-08  False              False
Current version (0.1.0)  1e-16  True               True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer

Weight decoupling enabled in AdaBelief
Rectification enabled in AdaBelief
1/50 (1m 55s) train loss: 1.744, train acc: 53.23%, val loss: 1.250, val acc: 63.23%
Validation accuracy in first epoch: 63.23%
Saving model...
2/50 (3m 50s) train loss: 1.183, train acc: 67.06%, val loss: 0.790, val acc: 77.06%
Validation accuracy is increasing: 63.23% --> 77.06%
Saving model...
3/50 (5m 44s) train loss: 1.016, train acc: 71.44%, val loss: 0.692, val acc: 79.93%
Validation accuracy is increasing: 77.06% --> 79.93%
Saving model...
4/50 (

In [59]:
path = 'new_transformer.pt'
model = torch.load(path)
_, test_acc = evaluateImageText(model, test_loader, criterion, device)
print('Accuracy in test set: %.2f%%' % (100 * test_acc))

Accuracy in test set: 97.18%


In [62]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NewTransformer(len(train_vocab2num), ResBasicBlock, [3, 4, 6, 3], n_encoders=1, max_seq=max_seq).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = AdaBelief(model.parameters(), lr=3e-04)
path = 'brandnew_transformer.pt'
n_epochs = 50
train_acc_list, val_acc_list = trainImageText(model, train_loader, val_loader, n_epochs, criterion, optimizer, device, path, data_aug=0.6)
model = torch.load(path)
_, test_acc = evaluateImageText(model, test_loader, criterion, device)
print('Accuracy in test set: %.2f%%' % (100 * test_acc))
n_epochs = 30
model = torch.load(path)
optimizer = AdaBelief(model.parameters(), lr=1e-04)
train_acc_list, val_acc_list = trainImageText(model, train_loader, val_loader, n_epochs, criterion, optimizer, device, path, data_aug=0)
model = torch.load(path)
_, test_acc = evaluateImageText(model, test_loader, criterion, device)
print('Accuracy in test set: %.2f%%' % (100 * test_acc))

Please check your arguments if you have upgraded adabelief-pytorch from version 0.0.5.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  ---------
adabelief-pytorch=0.0.5  1e-08  False              False
Current version (0.1.0)  1e-16  True               True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer

Weight decoupling enabled in AdaBelief
Rectification enabled in AdaBelief
1/50 (1m 55s) train loss: 1.756, train acc: 53.03%, val loss: 1.166, val acc: 67.53%
Validation accuracy in first epoch: 67.53%
Saving model...
2/50 (3m 53s) train loss: 1.188, train acc: 67.07%, val loss: 0.799, val acc: 78.49%
Validation accuracy is increasing: 67.53% --> 78.49%
Saving model...
3/50 (5m 48s) train loss: 1.023, train acc: 70.97%, val loss: 0.652, val acc: 81.68%
Validation accuracy is increasing: 78.49% --> 81.68%
Saving model...
4/50 (

In [34]:
class MobileRCNN(nn.Module):
    def __init__(self, n_vocab, output_size=27, embedding_size=512, n_layers=1, hidden_size=128, dropout=0.1):
        super(MobileRCNN, self).__init__()

        self.imagemodel = models.mobilenet_v2(pretrained=True)
        self.fc0 = nn.Linear(1000, embedding_size)
        self.embedding = nn.Embedding(n_vocab, embedding_size, padding_idx=1)
        self.lstm = nn.LSTM(input_size=embedding_size, hidden_size=hidden_size, num_layers=n_layers,
                            dropout=dropout, bidirectional=True, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.W = nn.Linear(embedding_size + 2 * hidden_size, hidden_size)
        self.tanh = nn.Tanh()
        self.fc = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, image, x):
        image = self.imagemodel(image)
        image = self.fc0(image)
        image = torch.unsqueeze(image, dim=1)
        x = self.embedding(x)
        x = x + image
        lstm_out, _ = self.lstm(x)  # lstm_out = (batch_size, seq_len, 2 * hidden_size)
        x = torch.cat([lstm_out, x], 2)
        linear_output = self.tanh(self.W(x))
        linear_output = linear_output.permute(0, 2, 1)  # linear_output = (batch_size, seq_len, hidden_size)
        max_out_features = F.max_pool1d(linear_output, linear_output.size()[2]).squeeze(2)
        max_out_features = self.dropout(max_out_features)
        out = self.fc(max_out_features)
        return out

In [78]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MobileRCNN(len(train_vocab2num), dropout=0.3).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = AdaBelief(model.parameters(), lr=3e-04)
path = 'mobilercnn.pt'
n_epochs = 20
train_acc_list, val_acc_list = trainImageText(model, train_loader, val_loader, n_epochs, criterion, optimizer, device, path, data_aug=0.6, fusion_ratio=0.05)
model = torch.load(path)
_, test_acc = evaluateImageText(model, test_loader, criterion, device)
print('Accuracy in test set: %.2f%%' % (100 * test_acc))

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:61: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Please check your arguments if you have upgraded adabelief-pytorch from version 0.0.5.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  ---------
adabelief-pytorch=0.0.5  1e-08  False              False
Current version (0.1.0)  1e-16  True               True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer

Weight decoupling enabled in AdaBelief
Rectification enabled in AdaBelief
1/20 (1m 57s) train loss: 1.376, train acc: 65.30%, val loss: 0.666, val acc: 83.21%
Validation accuracy in first epoch: 83.21%
Saving model...
2/20 (3m 54s) train loss: 0.682, train acc: 82.12%, val loss: 0.406, val acc: 89.27%
Validation accuracy is increasing: 83.21% --> 89.27%
Saving model...
3/20 (5m 51s) train loss: 0.488, train acc: 87.34%, val loss: 0.295, val acc: 92.28%
Validation accuracy is increasing: 89.27% --> 92.28%
Saving model...
4/20 (

In [79]:
n_epochs = 10
path = 'mobilercnn.pt'
model = torch.load(path)
optimizer = AdaBelief(model.parameters(), lr=1e-04)
train_acc_list, val_acc_list = trainImageText(model, train_loader, val_loader, n_epochs, criterion, optimizer, device, path, data_aug=0)
model = torch.load(path)
_, test_acc = evaluateImageText(model, test_loader, criterion, device)
print('Accuracy in test set: %.2f%%' % (100 * test_acc))

Please check your arguments if you have upgraded adabelief-pytorch from version 0.0.5.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  ---------
adabelief-pytorch=0.0.5  1e-08  False              False
Current version (0.1.0)  1e-16  True               True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer

Weight decoupling enabled in AdaBelief
Rectification enabled in AdaBelief
1/10 (1m 46s) train loss: 0.058, train acc: 98.88%, val loss: 0.086, val acc: 97.59%
Validation accuracy in first epoch: 97.59%
Saving model...
2/10 (3m 32s) train loss: 0.051, train acc: 99.02%, val loss: 0.078, val acc: 97.87%
Validation accuracy is increasing: 97.59% --> 97.87%
Saving model...
3/10 (5m 18s) train loss: 0.053, train acc: 98.87%, val loss: 0.076, val acc: 97.87%
4/10 (7m 4s) train loss: 0.051, train acc: 98.97%, val loss: 0.073, val acc

In [35]:
import torchvision.models as models

class VGGTextCNN(nn.Module):
    def __init__(self, n_vocab, embedding_size=512, output_size=27, kernel_num=100, kernel_size=[3, 4, 5], dropout=0.1):
        super(VGGTextCNN, self).__init__()

        self.imagemodel = models.vgg11_bn(pretrained=True)
        self.fc0 = nn.Linear(1000, embedding_size)
        V = n_vocab
        D = embedding_size
        C = output_size
        Ci = 1
        Co = kernel_num
        Ks = kernel_size

        self.embed = nn.Embedding(V, D, padding_idx=1)
        self.convs = nn.ModuleList([nn.Conv2d(Ci, Co, (K, D)) for K in Ks])
        self.dropout = nn.Dropout(dropout)
        self.fc1 = nn.Linear(len(Ks) * Co, 128)
        self.fc2 = nn.Linear(128, C)
        self.relu = nn.ReLU()

    def forward(self, image, x):
        image = self.imagemodel(image)
        image = self.fc0(image)
        image = torch.unsqueeze(image, dim=1)
        x = self.embed(x)  # (N, W, D)

        x = x + image
        x = x.unsqueeze(1)  # (N, Ci, W, D)

        x = [self.relu(conv(x)).squeeze(3) for conv in self.convs]  # [(N, Co, W), ...]*len(Ks)

        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]  # [(N, Co), ...]*len(Ks)

        x = torch.cat(x, 1)

        x = self.dropout(x)  # (N, len(Ks)*Co)
        x = self.relu(self.fc1(x))
        return self.dropout(self.fc2(x))

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = VGGTextCNN(len(train_vocab2num), dropout=0.3).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = AdaBelief(model.parameters(), lr=3e-04)
path = 'vggtext.pt'
n_epochs = 20
train_acc_list, val_acc_list = trainImageText(model, train_loader, val_loader, n_epochs, criterion, optimizer, device, path, data_aug=0.6, fusion_ratio=0.05)
model = torch.load(path)
_, test_acc = evaluateImageText(model, test_loader, criterion, device)
print('Accuracy in test set: %.2f%%' % (100 * test_acc))


Please check your arguments if you have upgraded adabelief-pytorch from version 0.0.5.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  ---------
adabelief-pytorch=0.0.5  1e-08  False              False
Current version (0.1.0)  1e-16  True               True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer

Weight decoupling enabled in AdaBelief
Rectification enabled in AdaBelief
1/20 (2m 28s) train loss: 1.835, train acc: 50.23%, val loss: 0.668, val acc: 83.67%
Validation accuracy in first epoch: 83.67%
Saving model...
2/20 (5m 0s) train loss: 1.232, train acc: 64.98%, val loss: 0.362, val acc: 91.07%
Validation accuracy is increasing: 83.67% --> 91.07%
Saving model...
3/20 (7m 27s) train loss: 1.002, train acc: 70.24%, val loss: 0.229, val acc: 94.63%
Validation accuracy is increasing: 91.07% --> 94.63%
Saving model...
4/20 (9

In [ ]:
n_epochs = 10
model = torch.load(path)
optimizer = AdaBelief(model.parameters(), lr=1e-04)
train_acc_list, val_acc_list = trainImageText(model, train_loader, val_loader, n_epochs, criterion, optimizer, device, path, data_aug=0)
model = torch.load(path)
_, test_acc = evaluateImageText(model, test_loader, criterion, device)
print('Accuracy in test set: %.2f%%' % (100 * test_acc))

Please check your arguments if you have upgraded adabelief-pytorch from version 0.0.5.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  ---------
adabelief-pytorch=0.0.5  1e-08  False              False
Current version (0.1.0)  1e-16  True               True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer

Weight decoupling enabled in AdaBelief
Rectification enabled in AdaBelief
1/10 (1m 43s) train loss: 0.749, train acc: 75.08%, val loss: 0.112, val acc: 97.32%
Validation accuracy in first epoch: 97.32%
Saving model...
2/10 (3m 27s) train loss: 0.717, train acc: 76.18%, val loss: 0.107, val acc: 97.41%
Validation accuracy is increasing: 97.32% --> 97.41%
Saving model...
3/10 (5m 12s) train loss: 0.713, train acc: 76.25%, val loss: 0.111, val acc: 97.64%
Validation accuracy is increasing: 97.41% --> 97.64%
Saving model...
4/10 (

In [36]:
from scipy.stats import mode

def predictByMode(model_list, model_usage, image, src, device):    
    pred = torch.zeros(image.size(0), len(model_list))
    for i, model in enumerate(model_list):
        model.eval()
        model = model.to(device)
        if model_usage[i]['img'] and model_usage[i]['src']:
            image = image.to(device)
            src = src.to(device)
            pred[:, i] = torch.argmax(model(image, src), dim=1)
        elif model_usage[i]['img']:
            image = image.to(device)
            pred[:, i] = torch.argmax(model(image), dim=1)
        else:
            src = src.to(device)
            pred[:, i] = torch.argmax(model(src), dim=1)
    
    pred = torch.from_numpy(mode(pred, axis=1)[0])
    return torch.squeeze(pred, dim=1).to(device)

In [37]:
def predictByWeight(model_list, model_usage, image, src, device, weights, num_classes=27):
    pred = torch.zeros(image.size(0), num_classes).to(device)
    for i in range(len(model_list)):
        model = model_list[i].to(device)
        model.eval()
        output = None
        if model_usage[i]['img'] and model_usage[i]['src']:
            image = image.to(device)
            src = src.to(device)
            output = model(image, src)
        elif model_usage[i]['img']:
            image = image.to(device)
            output = model(image)
        else:
            src = src.to(device)
            output = model(src)
        
        output = weights[i] * output
        pred.add_(output)
    
    return torch.argmax(pred, dim=1)

In [38]:
def evaluateEnsemble(model_list, model_usage, data, device, weights=None, num_classes=27):
    correct = 0
    total = 0
    for image, src, labels in data:
        pred = None
        labels = labels.to(device)
        if weights is None:
            pred = predictByMode(model_list, model_usage, image, src, device)
        else:
            pred = predictByWeight(model_list, model_usage, image, src, device, weights, num_classes)

        correct += (pred == labels).sum().item()
        total += len(labels)

    return correct / total

In [39]:
# Test
new_transformer = torch.load('new_transformer.pt')
googlenet = torch.load('googlenet.pt')
shufflenet = torch.load('shufflenet.pt')
mobilercnn = torch.load('mobilercnn.pt')
vggtext = torch.load('vggtext.pt')

model_list = [new_transformer, googlenet, shufflenet, mobilercnn, vggtext]
model_usage = [{'img': True, 'src': True},
               {'img': True, 'src': False},
               {'img': True, 'src': False},
               {'img': True, 'src': True},
               {'img': True, 'src': True}]

In [90]:
# Test
val_acc = evaluateEnsemble(model_list, model_usage, val_loader, device)
print('Ensemble accuracy in validation set: %.2f%%' % (100 * val_acc))
test_acc = evaluateEnsemble(model_list, model_usage, test_loader, device)
print('Ensemble accuracy in final test set: %.2f%%' % (100 * test_acc))

Ensemble accuracy in validation set: 98.47%
Ensemble accuracy in final test set: 97.83%


In [91]:
def generateRandomWeights(d, start=0.0, end=1.0):
    tmp = np.random.uniform(start, end, size=d)
    weights = [None] * len(tmp)
    for i in range(len(weights)):
        weights[i] = tmp[i] / np.sum(tmp)

    return weights

def findWeights(model_list, model_usage, train_data, val_data, device, k=1000, path='weights.pt'):
    weights = generateRandomWeights(len(model_list))
    tmp = weights.copy()
    val_acc = 0
    max_val_acc = 0
    train_acc = 0
    max_train_acc = 0
    for i in range(k):
        torch.cuda.empty_cache()
        train_acc = evaluateEnsemble(model_list, model_usage, train_data, device, tmp)
        print('Now %d/%d train acc: %.2f%%' % (i + 1, k, 100 * train_acc))
        if max_train_acc < train_acc:
            val_acc = evaluateEnsemble(model_list, model_usage, val_data, device, tmp)
            if max_val_acc < val_acc:
                max_train_acc = train_acc
                print('Now %d/%d max val acc increasing: %.2f%% -> %.2f%%' % (i + 1, k, 100 * max_val_acc, 100 * val_acc))
                max_val_acc = val_acc
                weights = tmp.copy()
                torch.save(weights, path)

        tmp = generateRandomWeights(len(model_list))

    return weights

In [114]:
# Test
start = time.time()
weights = findWeights(model_list, model_usage,  val_loader, test_loader, device, k=20)
print(weights)
print(timeSince(start))

Now 1/20 train acc: 98.01%
Now 1/20 max val acc increasing: 0.00% -> 97.83%
Now 2/20 train acc: 97.55%
Now 3/20 train acc: 98.52%
Now 4/20 train acc: 97.46%
Now 5/20 train acc: 98.20%
Now 5/20 max val acc increasing: 97.83% -> 97.97%
Now 6/20 train acc: 97.78%
Now 7/20 train acc: 98.29%
Now 7/20 max val acc increasing: 97.97% -> 98.01%
Now 8/20 train acc: 97.69%
Now 9/20 train acc: 98.43%
Now 10/20 train acc: 98.47%
Now 11/20 train acc: 98.24%
Now 12/20 train acc: 97.32%
Now 13/20 train acc: 98.33%
Now 14/20 train acc: 98.38%
Now 15/20 train acc: 97.46%
Now 16/20 train acc: 98.52%
Now 17/20 train acc: 98.15%
Now 18/20 train acc: 98.24%
Now 19/20 train acc: 98.52%
Now 20/20 train acc: 96.99%
[0.22262474308644636, 0.26835218907530883, 0.14642288040955073, 0.20878930780357, 0.153810879625124]
10m 3s


In [95]:
val_acc = evaluateEnsemble(model_list, model_usage, val_loader, device, weights)
print('Ensemble accuracy in validation set: %.2f%%' % (100 * val_acc))
test_acc = evaluateEnsemble(model_list, model_usage, test_loader, device, weights)
print('Ensemble accuracy in final test set: %.2f%%' % (100 * test_acc))

Ensemble accuracy in validation set: 98.47%
Ensemble accuracy in final test set: 97.92%


In [41]:
def makePredictions(model_list, model_usage, device, data=real_test_loader, weights=None):
    predictions = None
    for i, (image, src) in enumerate(data):
        if weights is None:
            pred = predictByMode(model_list, model_usage, image, src, device)
        else:
            pred = predictByWeight(model_list, model_usage, image, src, device, weights)
        
        if predictions is None:
            predictions = pred
        else:
            predictions = torch.cat((predictions, pred), dim=0)

    return predictions

In [42]:
start = time.time()
weights = torch.load('weights.pt')
predictions = makePredictions(model_list, model_usage, device, real_test_loader, weights)
print('%s' % timeSince(start))
pred = [None] * len(predictions)
for i in range(len(predictions)):
    pred[i] = num2cate[predictions[i].item()]

id_data = test_csv['id']
predict = pd.DataFrame(data=id_data)
predict['category'] = pred
predict

3m 10s


,id,category
0,26266,Innerwear
1,22134,Topwear
2,28358,Belts
3,15554,Shoes
4,53408,Innerwear
...,...,...
21623,39737,Eyewear
21624,57477,Topwear
21625,22312,Topwear
21626,54105,Lips


In [43]:
predict.to_csv('predict.csv', index=False)

In [44]:
from google.colab import files

files.download('predict.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>